In [197]:
#import dependencies 
import requests
import json
import pandas as pd

In [198]:
#connect to the API (rapid api)
url = "https://live-golf-data.p.rapidapi.com/leaderboard"

querystring = {"tournId":"014","year":"2022"}
headers = {
	"X-RapidAPI-Host": "live-golf-data.p.rapidapi.com",
	"X-RapidAPI-Key": "5f629a65d7msh1010ca075b711c3p1ace3cjsndaf17672e132"
}

response = requests.get(url, headers=headers, params=querystring).json()
#find the main dictionary key, in this case it's leaderboardrows
response['leaderboardRows']

[{'lastName': 'Scheffler',
  'firstName': 'Scottie',
  'playerId': '46046',
  'courseId': '014',
  'status': 'active',
  'total': '-10',
  'currentRoundScore': '-1',
  'position': '1',
  'totalStrokesFromCompletedRounds': '278',
  'roundComplete': True,
  'rounds': [{'scoreToPar': '-3',
    'roundId': {'$numberInt': '1'},
    'courseId': '014',
    'courseName': 'Augusta National Golf Club',
    'strokes': {'$numberInt': '69'}},
   {'scoreToPar': '-5',
    'roundId': {'$numberInt': '2'},
    'courseId': '014',
    'courseName': 'Augusta National Golf Club',
    'strokes': {'$numberInt': '67'}},
   {'scoreToPar': '-1',
    'roundId': {'$numberInt': '3'},
    'courseId': '014',
    'courseName': 'Augusta National Golf Club',
    'strokes': {'$numberInt': '71'}},
   {'scoreToPar': '-1',
    'roundId': {'$numberInt': '4'},
    'courseId': '014',
    'courseName': 'Augusta National Golf Club',
    'strokes': {'$numberInt': '71'}}],
  'startingHole': {'$numberInt': '1'},
  'currentHole': {'$

In [199]:
#pull data on another dictionary key to find the information on the specific round
response['leaderboardRows'][0]['rounds'][0]

{'scoreToPar': '-3',
 'roundId': {'$numberInt': '1'},
 'courseId': '014',
 'courseName': 'Augusta National Golf Club',
 'strokes': {'$numberInt': '69'}}

In [200]:
cut_Score = int(response['cutLines'][0]['cutScore'])

In [201]:
current_round = response['roundId']['$numberInt']

In [202]:
df = pd.DataFrame(columns=["position","full_name","total", "current_round","currentRoundScore","cut_score","player_id","status","strokes","roundId","round_score"])

In [203]:
for data in response['leaderboardRows']:
    first_name = data['firstName']
    last_name = data['lastName']
    position = data['position']
    player_id = float(data['playerId'])
    total = data['total']
    currentroundscore = data['currentRoundScore']
    status = data['status']
    full_name = first_name +' '+ last_name
    #comment out below before 1st round completes
    for rounds in data['rounds']:
        round_Id = rounds['roundId']['$numberInt']
        round_score = rounds['scoreToPar']
        strokes = rounds['strokes']['$numberInt']
        
        df= df.append({'position':position, 'full_name':full_name,'player_id':player_id, 'total':total, 'currentRoundScore':currentroundscore, 'cut_score':cut_Score, 'current_round':current_round, 'status':status,'roundId':round_Id, 'round_score':round_score, 'strokes':strokes}, ignore_index=True)
df['total']=df['total'].replace('E',0)
df['round_score']= df['round_score'].replace('E',0)
#df['round_score']= pd.to_numeric(df['round_score'])




In [205]:
df['round_score']= pd.to_numeric(df['round_score'])
df['round_score']

0      -3
1      -5
2      -1
3      -1
4       1
       ..
278     5
279    12
280     9
281    13
282     4
Name: round_score, Length: 283, dtype: int64

In [206]:
df['g']=df.groupby(['full_name','player_id']).cumcount().add(1)
df3=(pd.pivot_table(df,index=['full_name','player_id'],
                    values='round_score',
                    columns=['roundId'],
                    aggfunc='max'
                ))
df3

,roundId,1,2,3,4
full_name,player_id,,,,
Aaron Jarvis,61560.0,9.0,2.0,NaN,NaN
Abraham Ancer,45526.0,0.0,7.0,NaN,NaN
Adam Scott,24502.0,2.0,2.0,8.0,2.0
Austin Greaser,61143.0,2.0,5.0,NaN,NaN
Bernhard Langer,1666.0,4.0,4.0,NaN,NaN
...,...,...,...,...,...
Viktor Hovland,46717.0,0.0,4.0,-1.0,1.0
Webb Simpson,29221.0,-1.0,2.0,1.0,4.0
Will Zalatoris,47483.0,-1.0,0.0,3.0,-5.0


In [96]:
df.to_csv('leaderboard.csv', encoding='utf-8', index=False)